### differential-lattice running in p55py (a jupyter widget)


In [1]:
# imports
from p55py import P55Wrapper
from numpy import array
from modules.differentialLattice import DifferentialLattice
from modules.helpers import spawn_circle

In [2]:
# differential-lattice constants
colors = {
  'back': (255,255,255,255),
  'front': (0,0,0,178),
  'cyan': (0,153,153,178),
  'purple': (153,0,153,178),
  'light': (0,0,0,51),
}

threads = 512
zone_leap = 512

size = 320
one = 1.0/size

export_steps = 5
render_steps = 5

init_num = 20

line_width = one*2.5

stp = one*0.03
spring_stp = 5
reject_stp = 0.1
cohesion_stp = 1.0

max_capacity = 30

node_rad = 4*one
spring_reject_rad = node_rad*1.9
spring_attract_rad = node_rad*2.0
outer_influence_rad = 10.0*node_rad
link_ignore_rad = 0.5*outer_influence_rad

### define three p55py callbacks: setup, step, and render

In [3]:
# setup initializes the model
def setup_dl(self):
  self.dl = DifferentialLattice(
      size,
      stp,
      spring_stp,
      reject_stp,
      cohesion_stp,
      max_capacity,
      node_rad,
      spring_reject_rad,
      spring_attract_rad,
      outer_influence_rad,
      link_ignore_rad,
      threads = threads,
      zone_leap = zone_leap,
      nmax = 50000000
      )
  spawn_circle(self.dl, init_num , xy=array([[0.5,0.5]]), dst=node_rad*0.8, rad=0.01)
    
# step updates the model
def step_dl(self):
    self.dl.step()
    self.dl.spawn(ratio=0.1, age=1000)

# render draws the model (idempotent)
def render_dl(self):
    # clear the screen
    self.fill(255)
    self.cur_outline = self.cur_fill
    self.rectangle(0,0,320,320)

    # get the geometry to draw
    num = self.dl.num
    vertices, edges = self.dl.link_export()
    # window size is currently 320
    vertices = vertices * 320

    # draw edges
    self.cur_fill = colors['purple']
    for a,b in edges:
      self.line(vertices[a,0], vertices[a,1], vertices[b,0], vertices[b,1])

    # draw nodes
    self.cur_fill = colors['front']
    for i in xrange(num):
      self.ellipse(vertices[i,0], vertices[i,1], dl.node_rad*0.6, fill=True)

In [4]:
wrapper = P55Wrapper(setupfn=setup_dl, stepfn=step_dl, renderfn=render_dl)
wrapper.play.max = 500
wrapper.redrawFreq = 5  # draw only every 5th step
wrapper.widget